In [ ]:
  # import libraries
try:
    # %tensorflow_version only exists in Colab.
    !pip install tf-nightly
except Exception:
    pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/














































Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/






















2.12.0-dev20221230


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-12-30 20:50:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv

Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...

Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.

HTTP request sent, awaiting response... 200 OK

Length: 358233 (350K) [text/tab-separated-values]

Saving to: ‘train-data.tsv.6’



train-data.tsv.6    100%[===================>] 349.84K  --.-KB/s    in 0.04s   



2022-12-30 20:50:17 (8.88 MB/s) - ‘train-data.tsv.6’ saved [358233/358233]



--2022-12-30 20:50:17--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...

Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.

HTTP request sent, awaiting response... 200 OK

Length: 118774 (116K) [text/tab-separated-values]

Saving to: ‘valid-data.tsv.6’



valid-data.

In [ ]:
traindata = pd.read_table(train_file_path)
vailddata = pd.read_table(test_file_path)

In [ ]:
vailddata.head()

,ham,i am in hospital da. . i will return home in evening
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [ ]:
#i think by doing columns this way i lose the first entry but well its not a big deal i think
traindata.columns = ['Classification', 'message']
vailddata.columns = ['Classification', 'message']

traindata.head()

,Classification,message
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [ ]:
fulldata = traindata.message.append(vailddata.message, ignore_index=True) #i combine both datasets because i want to fill tokenizer (which i explain later) on entire data


In [ ]:
a = fulldata.str.split().apply(len) #Later i will need to know how many words emailes have, because i need them to have the same number of words i think good idea is to take mean number of words and trim excess words

In [ ]:
print(max(a))
print(a.quantile(0.75)) #actually i decided to go with 3th quantile, where 75% of data is below this number and only 25% is excess

length_of_emails = int(a.quantile(0.75))

171

23.0


In [ ]:
print(len(fulldata)) #not important but we have 5569 messages in total 

5569


In [ ]:
from keras.preprocessing.text import Tokenizer

# Initialize tokenizer, what this does is essentialy it maps each word to some number, so we get a full vocabulary of numbers where each is some word. its important cuz you can,t do anything with text strings
tokenizer = Tokenizer()

# Fit the Tokenizer on the entire data so it has all the words in vocab
tokenizer.fit_on_texts(fulldata)

# Convert the training data to sequences of tokens
train_X = tokenizer.texts_to_sequences(traindata.message)

# Convert the validation data to sequences of tokens
val_X = tokenizer.texts_to_sequences(vailddata.message)

In [ ]:
print(val_X[0]) #it looks like this
print(vailddata.message.loc[0])

[25, 122, 37, 116, 7819, 50, 742, 3]

not much, just some textin'. how bout you?


In [ ]:
#now i use this method to trim any email that has more than a mean amount of words in data (16) , padding fills this array of emails as numbers with 0, where 'post' means fill at the end if its less than 16. 
#truncating trims emails with more than 16 words, again post meaning trim at the end.  'pre' would trim at the beginning.
train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=length_of_emails, padding='post', truncating='post') 
val_X =  keras.preprocessing.sequence.pad_sequences(val_X, maxlen=length_of_emails, padding='post', truncating='post')

In [ ]:
print(val_X) #weird but ok

[[  25  122   37 ...    0    0    0]

 [   1  388  569 ...    0    0    0]

 [1683  138    4 ...    0    0    0]

 ...

 [ 131   22    3 ...    4 8993   20]

 [  33  158  856 ... 1686 8994 1020]

 [  25  958   47 ... 1083 4419   57]]


In [ ]:
vocab_size = len(tokenizer.word_index) + 1 #soo here is how to get vocabulary size out of this tokenizer. as you see it has more entries than fulldata words but this is because it has all the unique words, earlier we counted
#just a total number of words
print(vocab_size)

8995


In [ ]:

train_Y = traindata.Classification #remove labels from training and validation data
val_Y = vailddata.Classification

In [ ]:
val_Y.replace(to_replace = ['ham','spam'], value = [0,1],inplace = True) #now labels are string, again they need to be numbers so replace ham with 0 and spam with 1
train_Y.replace(to_replace = ['ham','spam'], value = [0,1], inplace = True)
train_Y.head()

0    0
1    0
2    0
3    0
4    0
Name: Classification, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split #its good to have a test data so i take a small fraction of train data to get test data by using this method, pretty self explainatory.
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, random_state=2, test_size = 0.1) 

In [ ]:
print(len(train_X))
print(len(test_X))
print(len(val_X))

3760

418

1391


In [ ]:
#okay the model, 
model = keras.Sequential([
    
  keras.layers.Embedding(input_dim=vocab_size, output_dim=75, input_length=length_of_emails), #generally it preprocess the data so the integers are mapped into vectors , output dim lets you choose how many numbers this vector has
  keras.layers.LSTM(units=256, dropout=0.3, recurrent_dropout=0.3), #
  keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=[ 'acc'])
model.summary()

Model: "sequential_2"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 embedding_2 (Embedding)     (None, 23, 75)            674625    

                                                                 

 lstm_2 (LSTM)               (None, 256)               339968    

                                                                 

 dense_2 (Dense)             (None, 1)                 257       

                                                                 


Total params: 1,014,850

Trainable params: 1,014,850

Non-trainable params: 0

_________________________________________________________________


In [ ]:
model.fit(train_X, train_Y, epochs=10, batch_size = 32,validation_data=(val_X, val_Y)) #train the data

Epoch 1/10

118/118 [==============================] - 39s 297ms/step - loss: 0.1533 - acc: 0.9492 - val_loss: 0.0681 - val_acc: 0.9799

Epoch 2/10

118/118 [==============================] - 27s 229ms/step - loss: 0.0491 - acc: 0.9883 - val_loss: 0.0615 - val_acc: 0.9842

Epoch 3/10

118/118 [==============================] - 30s 251ms/step - loss: 0.0088 - acc: 0.9981 - val_loss: 0.0636 - val_acc: 0.9885

Epoch 4/10

118/118 [==============================] - 29s 245ms/step - loss: 0.0142 - acc: 0.9955 - val_loss: 0.0564 - val_acc: 0.9878

Epoch 5/10

118/118 [==============================] - 26s 224ms/step - loss: 0.0070 - acc: 0.9981 - val_loss: 0.0764 - val_acc: 0.9871

Epoch 6/10

118/118 [==============================] - 26s 225ms/step - loss: 0.0029 - acc: 0.9997 - val_loss: 0.0677 - val_acc: 0.9878

Epoch 7/10

118/118 [==============================] - 27s 227ms/step - loss: 0.0026 - acc: 0.9997 - val_loss: 0.0770 - val_acc: 0.9871

Epoch 8/10

118/118 [====================

In [ ]:
y_pred = model.predict(test_X) #predictions on test data to see how good model is later
y_prediction = []
for i in y_pred:
  if i > 0.3:
    y_prediction.append(1)
  else:
    y_prediction.append(0)


14/14 [==============================] - 1s 53ms/step


In [ ]:
print(y_prediction)
print(test_Y)

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# so this function allows me to see precision and recall
#precision is basically number of  true positive (predictions belonging to class closer to 1 in binary classification , which are correct)/all the positives model made eg. 80/100 means 80% precision and that 80 of 100 predictions for eg of spam were correct.
#recall is  number of true positves but this time divided by an actual number of positives so lets say we have 100 spam emails. if model predicts 70 of them then recall is 70/100 = 70%
#f1 score is just some math including previous metrics considered as good indicator 
precision, recall, f1_score, support = precision_recall_fscore_support(test_Y, y_prediction, average='binary')

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')

F1 = 2 * (precision * recall) / (precision + recall)
print(f'F1 score the (higher the better between 0 and 1) is:  {f1_score:.3f}')

#seems good

#IMPORTANT, due to this task making me predict specific emails, i see that when i set the threshold of whether an email is a spam or not to 0.3 i cannot complete the task but it typically has a better F1 score.
#when i set it to 0.0001 the task is completed but the F1 score drops

Precision: 0.917

Recall: 0.948

F1 score the (higher the better between 0 and 1) is:  0.932


In [ ]:

pred_text = ["how are you doing today?"] #it needs to be in a list or tokenizer encodes each letter
pred_text  = tokenizer.texts_to_sequences(pred_text)
#pred_text = [[elem for sublist in pred_text for elem in sublist]] it flattened some version of a list no longer neccasary
print(pred_text)
pred_text = keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=16, padding='post', truncating='post') #each text to predict has to be trimmed aswell

print(pred_text)

#just testing how to do function they want same code below 

[[50, 22, 3, 159, 89]]

[[ 50  22   3 159  89   0   0   0   0   0   0   0   0   0   0   0]]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text  = tokenizer.texts_to_sequences([pred_text])

  #pred_text = [[elem for sublist in pred_text for elem in sublist]]
  print(pred_text)
  pred_text = keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=16, padding='post', truncating='post')

  #print(pred_text)

  prediction = model.predict(pred_text)
  #print(prediction) its in a list in a list (2d list) thats why double index to take the value below
  if prediction[0][0] > 0.3: #this threshold seems to work the best (0.3)
    label = 'spam'
  else:
    label = 'ham'
  print(prediction[0][0], label) 
  return (prediction[0][0], label)

pred_text = "hello your computer has virus pay 10 dollars to get this for free" 
prediction = predict_message(pred_text)
print(prediction)

[[258, 13, 932, 118, 394, 431, 1692, 2, 31, 40, 12, 48]]

1/1 [==============================] - 0s 152ms/step

0.103817336 ham

(0.103817336, 'ham')


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[[50, 22, 3, 159, 89]]

1/1 [==============================] - 0s 27ms/step

7.5539596e-05 ham

[[1248, 89, 2, 84, 469, 16]]

1/1 [==============================] - 0s 31ms/step

0.99081236 spam

[[1, 93, 70, 2, 49, 28, 39, 261, 14, 4, 1068, 64, 663, 408]]

1/1 [==============================] - 0s 28ms/step

0.0027869695 ham

[[92, 102, 97, 375, 230, 9, 287, 37, 8433, 18, 13, 111, 2, 345, 379]]

1/1 [==============================] - 0s 31ms/step

0.9935156 spam

[[3, 17, 185, 370, 158, 16, 2, 128, 13, 152]]

1/1 [==============================] - 0s 33ms/step

0.993478 spam

[[79, 498, 14, 151, 98, 665, 5, 2287]]

1/1 [==============================] - 0s 28ms/step

2.7634576e-05 ham

[[926, 9, 13, 4283, 530, 20, 728, 2, 10, 74, 63, 134]]

1/1 [==============================] - 0s 26ms/step

4.8516435e-05 ham

You passed the challenge. Great job!
